In [1]:
# # Parameters
# detector = True
# fcg_path_benign='/mnt/E/re/Embedding vector generation based on function call graph for effective malware detection and classification/FCGs/benign'
# fcg_path_malware= '/mnt/E/re/Embedding vector generation based on function call graph for effective malware detection and classification/FCGs/malware'
# w2v_path='./model_params/word2vec.wordvectors'

In [13]:
import os
import pandas as pd
import pickle

def count_pickle_files(directory):
    count = 0
    for root, dirs, files in os.walk(directory):
        for file in files:
            if file.endswith('.pickle'):
                count += 1
    return count

def load_pickle(file_path):
    with open(file_path, 'rb') as file:
        return pickle.load(file)

def create_dataframe(directory, label):
    data = []
    for root, dirs, files in os.walk(directory):
        for file in files:
            if file.endswith('.pickle'):
                file_path = os.path.join(root, file)
                file_data = load_pickle(file_path)
                data.append({
                    'filename': file,
                    'label': label,
                    'file_path': file_path
                })
    return pd.DataFrame(data)

In [14]:
import os
import random
import json
import pandas as pd
import dask.dataframe as dd

def list_json_files(directory):
    json_files = []
    for root, _, files in os.walk(directory):
        for file in files:
            if file.endswith('.json'):
                json_files.append(os.path.join(root, file))
    return json_files

def sample_json_files(json_files, sample_size):
    return random.sample(json_files, sample_size)

def process_json_files(file_paths, label):
    data = []
    for file_path in file_paths:
        try:
            with open(file_path, 'r') as f:
                json_data = json.load(f)
                json_data['label'] = label
                json_data['file_path'] = file_path
                data.append(json_data)
        except Exception as e:
            print(f"Error processing file {file_path}: {e}")
    return pd.DataFrame(data)

def main():
    base_dir = '/mnt/E/mnt/bigDisk/yishan/dataset_disassemble/results'
    benign_dir = os.path.join(base_dir, 'benign')
    malware_dir = os.path.join(base_dir, 'malware')

    benign_files = list_json_files(benign_dir)
    malware_files = list_json_files(malware_dir)

    # Randomly sample 15,000 files from each directory
    sample_size = 30000
    sampled_benign_files = sample_json_files(benign_files, sample_size)
    sampled_malware_files = sample_json_files(malware_files, sample_size)

    # Process the sampled files
    benign_df = process_json_files(sampled_benign_files, 0)
    malware_df = process_json_files(sampled_malware_files, 1)

    combined_df = dd.from_pandas(pd.concat([benign_df, malware_df], ignore_index=True), npartitions=10)
    
    # Compute the combined DataFrame
    final_df = combined_df.compute()
    print(final_df)

if __name__ == '__main__':
    main()


ValueError: Sample larger than population or is negative

- My understanding of Normalization is based on the goals mentioned in the paper, which include the following:
1. Remove all comments: In radare2, only ; is used for comments. Refer to 7.1 Adding Metadata for more information.
2. Replace all numeric constant values with "N": Use re.sub() to iterate through all numeric constants with the regular expression \b0x[a-fA-F0-9]+\b|\b\d+\b.
3. Replace all irregular strings with "M": Use re.sub() to iterate through all irregular strings with the regular expression \b[^a-zA-Z0-9]+\b.
4. Replace all function names with their short names: Use re.sub() to iterate through all common function names with the regular expression \b(sub|loc|str|reloc|obj)_[0-9a-fA-F]+\b.
5. Connect the opcode and operand with "-":

In [ ]:
import re

class NormalizeAssembly:
    def __init__(self) -> None:
        pass

    def normalize(self, inst):
        """
        Normalize a given assembly instruction. According to the paper should have:
        1. Remove all comments;
        2. Replace all numeric constant values with "N";
        3. Replace all irregular strings with "M";
        4. Replace all function names with their short name;
           For example, replace "sub_406492" with "sub", replace "loc_100080CF" with "loc".
        5. Connect the opcode and operand with "-"
        """
        # Remove comments
        inst = re.sub(r';.*', '', inst)

        # Replace numeric constant values with "N"
        inst = re.sub(r'\b0x[0-9a-fA-F]+\b|\b\d+\b', 'N', inst)

        # Replace function names with their short name
        inst = re.sub(r'\b(sub|loc|str|reloc|obj)_[0-9a-fA-F]+\b', r'\1', inst)

        # Split the instruction into parts to process irregular strings
        asm_normed = []
        parts = re.split(r'(\s+)', inst)

        for part in parts:
            if re.match(r'\b[^a-zA-Z0-9]+\b', part):
                part = 'M'
            asm_normed.append(part)

        # Join the parts back together
        normalized_inst = ''.join(asm_normed)

        # Connect opcode and operand with "-"
        inst_parts = normalized_inst.split()
        if len(inst_parts) > 1:
            normalized_inst = f"{inst_parts[0]}-{' '.join(inst_parts[1:])}"
        else:
            normalized_inst = inst_parts[0]

        return normalized_inst

def normalize_dataframe(df, normalizer):
    df['data'] = df['data'].apply(lambda instructions: [normalizer.normalize(inst) for inst in instructions] if isinstance(instructions, list) else instructions)
    return df

def main():
    normalizer = NormalizeAssembly()


My understanding is that CBOW is used for predicting a word from its context, while Skip-gram does the opposite. Thus, the inputs and outputs are reversed for these two models. In the end, a softmax function is used to predict a probability distribution.

- vector_size: The dimensionality of the word vectors. The default value is 100. Generally, a larger vector size is better but requires more computational resources.
- window: The size of the context window, which determines the number of words to consider around the target word. The default value is 5.
- alpha: The learning rate. The default value is 0.025. The learning rate gradually decreases during training.
- min_count: Ignores all words with a frequency lower than this value. The default value is 5.
- sg: The training algorithm choice. 1 means Skip-gram is used, while 0 means CBOW is used. The default value is 0.
- hs: Indicates whether hierarchical softmax is used. The default value is 0, meaning it is not used.
- negative: The number of negative samples to use when negative sampling is employed. The default value is 5.
- epochs: The number of training iterations. The default value is 5

I spent some time on hyperparameter adjustment. After reading this paper Understanding the difficulty of training deep feedforward neural networks, I chose to use Xavier initialization for the model rather than importing the model from Gensim.

In [ ]:
import re
import numpy as np
from sklearn.preprocessing import OneHotEncoder

class Word2VecCBOW:
    def __init__(self, vocab_size, hidden_dim):
        # Initialize weights with Xavier initialization
        self.vocab_size = vocab_size
        self.hidden_dim = hidden_dim
        self.W1 = np.random.randn(vocab_size, hidden_dim) / np.sqrt(vocab_size / 2)
        self.W2 = np.random.randn(hidden_dim, vocab_size) / np.sqrt(hidden_dim / 2)

    def softmax(self, x):
        exp_x = np.exp(x - np.max(x))
        return exp_x / np.sum(exp_x, axis=0)

    def forward(self, X):
        self.h = np.dot(X, self.W1)
        self.u = np.dot(self.h, self.W2)
        self.y_hat = self.softmax(self.u)
        return self.y_hat

    def backward(self, X, y, learning_rate):
        error = self.y_hat - y
        dW2 = np.dot(self.h.T, error)
        dW1 = np.dot(X.T, np.dot(error, self.W2.T))
        self.W2 -= learning_rate * dW2
        self.W1 -= learning_rate * dW1

    def train(self, sentences, learning_rate=0.01, epochs=10):
        # Preprocess sentences to one-hot encoding
        word2index = {word: i for i, word in enumerate(set(sum(sentences, [])))}
        one_hot_encoder = OneHotEncoder(categories=[list(word2index.values())])
        for epoch in range(epochs):
            for sentence in sentences:
                for i, word in enumerate(sentence):
                    context = [word2index[w] for w in sentence[max(0, i - 2):i] + sentence[i + 1:i + 3]]
                    target = word2index[word]
                    X = one_hot_encoder.fit_transform([[context]]).toarray()
                    y = one_hot_encoder.fit_transform([[target]]).toarray().flatten()
                    self.forward(X)
                    self.backward(X, y, learning_rate)

    def get_word_vectors(self):
        return self.W1
